# SIPONGI BMKG Sumatera Dataset Analysis 2025

This notebook analyzes the structure and content of wildfire datasets from various provinces in Sumatera for 2025 (January-May).

In [74]:
# Import necessary libraries
import os
import pandas as pd
from IPython.display import display

# Display settings for pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [80]:
# List of province names
provinces = [
    'bangka-belitung',
    'bengkulu',
    'jambi',
    'kepulauan-riau',
    'riau',
    'sumatera-barat',
    'sumatera-selatan',
    'sumatera-utara',
]

# Read SiPongi files
sipongi = {}
sipongi_files = [f"{province}.xlsx" for province in provinces]
for file in sipongi_files:
    df_temp = pd.read_excel(file, sheet_name='Detail')
    sipongi[os.path.splitext(file)[0]] = df_temp

# Sort each SiPongi dataframe by the Tanggal column
for province in sipongi:
    if 'Tanggal' in sipongi[province].columns:
        # Convert to datetime if it's not already
        if not pd.api.types.is_datetime64_dtype(sipongi[province]['Tanggal']):
            sipongi[province]['Tanggal'] = pd.to_datetime(sipongi[province]['Tanggal'], format='%d-%m-%Y', errors='coerce')
        
        # Sort by Tanggal
        sipongi[province] = sipongi[province].sort_values(by='Tanggal').reset_index(drop=True)

        # Change Tanggal datetime format to 'dd-mm-yyyy'
        sipongi[province]['Tanggal'] = sipongi[province]['Tanggal'].dt.strftime('%d-%m-%Y')

display(sipongi)

# Print data structure information for each province in sipongi
# for province in sipongi:
#     print(f"Province: {province}")
#     print(f"Type: {type(sipongi[province])}")
#     print("Columns:", sipongi[province].columns.tolist())
#     print("Data types:\n", sipongi[province].dtypes)
#     print("Shape:", sipongi[province].shape)
#     print("-" * 40)

{'bangka-belitung':                       Provinsi             Kab Kota          Kecamatan               Desa     Tanggal      Waktu      Satelit Confidence  Latitude  Longitude
 0    Kepulauan Bangka Belitung       BANGKA SELATAN       Lepar Pongok            Penutuk  17-01-2025  12:54 WIB    NASA-SNPP     Medium  -2.96063  106.73106
 1    Kepulauan Bangka Belitung         BANGKA BARAT              Jebus       Sungai Buluh  31-01-2025  13:31 WIB    NASA-SNPP     Medium  -1.69669  105.36665
 2    Kepulauan Bangka Belitung               BANGKA            Belinyu     Remodong Indah  31-01-2025  13:54 WIB  NASA-NOAA20     Medium  -1.54117  105.71967
 3    Kepulauan Bangka Belitung               BANGKA        Mendo Barat            Penagan  01-02-2025  13:35 WIB  NASA-NOAA20     Medium  -2.27386  105.82907
 4    Kepulauan Bangka Belitung               BANGKA         Riau Silip               Riau  01-02-2025  13:35 WIB  NASA-NOAA20     Medium  -1.69441  105.87139
 5    Kepulauan Bangka Beli

In [79]:
# Read BMKG files
months = {'januari': 31, 'februari': 28, 'maret': 31, 'april': 30, 'mei': 26}
bmkg = {}
bmkg_files = [f"{province}/{province}-{month}.xlsx" for province in provinces for month in months]

for file in bmkg_files:
    province = file.split('/')[0]
    month = file.split('-')[-1].split('.')[0]
    df_temp = pd.read_excel(file, header=7, nrows=months[month])
    # bmkg[province][month] = df_temp
    if province not in bmkg:
        bmkg[province] = []
    bmkg[province].append(df_temp)

# After the loop, concatenate all month DataFrames for each province
for province in bmkg:
    bmkg[province] = pd.concat(bmkg[province], ignore_index=True)
    # Change TANGGAL datetime format to 'dd-mm-yyyy'
    if 'TANGGAL' in bmkg[province].columns:
        bmkg[province]['TANGGAL'] = pd.to_datetime(bmkg[province]['TANGGAL'], format='%d-%m-%Y', errors='coerce')
        bmkg[province]['TANGGAL'] = bmkg[province]['TANGGAL'].dt.strftime('%d-%m-%Y')

display(bmkg)

# Print data structure information for each province in bmkg
# for province in bmkg:
#     print(f"Province: {province}")
#     print(f"Type: {type(bmkg[province])}")
#     print("Columns:", bmkg[province].columns.tolist())
#     print("Data types:\n", bmkg[province].dtypes)
#     print("Shape:", bmkg[province].shape)
#     print("-" * 40)

{'bangka-belitung':         TANGGAL    TN    TX  TAVG  RH_AVG      RR   SS  FF_X  DDD_X  FF_AVG DDD_CAR
 0    01-01-2025  25.2  28.8  27.0      91     1.5  3.2     5    350       2       N
 1    02-01-2025  25.0  31.2  27.1      89     1.6  0.0     8    350       3       N
 2    03-01-2025  24.8  30.2  26.2      91     0.0  6.7    10    300       2      NW
 3    04-01-2025  23.9  30.0  26.9      86     0.0  1.5     4    300       3      NW
 4    05-01-2025  24.2  30.5  25.9      95     0.0  5.8     8    320       2       N
 5    06-01-2025  23.3  28.7  25.0      95    56.4  3.0     8    280       2      NW
 6    07-01-2025  23.4  29.0  26.4      91    24.3  0.0    12    350       5      NW
 7    08-01-2025  24.2  29.1  27.0      87    14.0  3.4    10    330       4       N
 8    09-01-2025  25.3  30.5  27.0      90     1.3  1.8     9    340       3       N
 9    10-01-2025  24.6  25.7  25.1      98    20.7  2.4     4    260       2       C
 10   11-01-2025  24.1  30.8  26.9      86   1

In [81]:
# Combine SiPongi and BMKG data for each province with a common key Tanggal. discard bmkg data that doesn't have matching Tanggal in sipongi. The column name for sipongi is 'Tanggal' and for bmkg is 'TANGGAL'
final = {}

for province in provinces:
    if province in sipongi and province in bmkg:
        # Ensure both columns are datetime for merging
        sipongi[province]['Tanggal'] = pd.to_datetime(sipongi[province]['Tanggal'], format='%d-%m-%Y', errors='coerce')
        bmkg[province]['TANGGAL'] = pd.to_datetime(bmkg[province]['TANGGAL'], format='%d-%m-%Y', errors='coerce')
        
        # Merge on Tanggal and TANGGAL
        merged_df = pd.merge(sipongi[province], bmkg[province], left_on='Tanggal', right_on='TANGGAL', how='inner')
        
        # Drop the TANGGAL column from the merged DataFrame
        merged_df.drop(columns=['TANGGAL'], inplace=True)

        # Store the merged DataFrame in a new variable 'merged'
        merged = merged_df.copy()
        final[province] = merged

# Combine all merged DataFrames into a single DataFrame
final = pd.concat(final.values(), ignore_index=True)

print(final)  # Display the first few rows of the merged data for the first province

/var/folders/jl/c3b0jzhs0px_szp3y8w8l6qc0000gn/T/ipykernel_7076/3053428593.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final = pd.concat(final.values(), ignore_index=True)


                       Provinsi                    Kab Kota                    Kecamatan                                   Desa    Tanggal      Waktu      Satelit Confidence  Latitude  Longitude    TN      TX  TAVG  RH_AVG      RR   SS  FF_X  DDD_X  FF_AVG DDD_CAR
0     Kepulauan Bangka Belitung              BANGKA SELATAN                 Lepar Pongok                                Penutuk 2025-01-17  12:54 WIB    NASA-SNPP     Medium  -2.96063  106.73106  24.9    29.8  25.8      96     1.1  4.0   8.0  350.0       2       N
1     Kepulauan Bangka Belitung                BANGKA BARAT                        Jebus                           Sungai Buluh 2025-01-31  13:31 WIB    NASA-SNPP     Medium  -1.69669  105.36665  24.6    29.5  26.9      83     0.6  1.0   9.0  340.0       5       N
2     Kepulauan Bangka Belitung                      BANGKA                      Belinyu                         Remodong Indah 2025-01-31  13:54 WIB  NASA-NOAA20     Medium  -1.54117  105.71967  24.6    2

In [78]:
# Convert final to final.csv
final.to_csv('final.csv', index=False)